In [1]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()


c:\Python310\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
def list_diseases():
    query = "SELECT DISTINCT trait_reported FROM `thegwensproject.genomiX.disease_variant_gene` order by trait_reported;"
    query_job = client.query(query)  # Make an API request.
    return query_job.to_dataframe()

In [ ]:
def count_variants(chr, pos):
    if pos == 0:
        query = (
            "SELECT chr_id_b37, position_b37, COUNT(*) as count FROM thegwensproject.genomiX.variants WHERE chr_id_b37 = '" +str(chr) +"' GROUP BY chr_id_b37, position_b37 HAVING COUNT(*) > 1 LIMIT 20;")
    else:
        query = (
            "SELECT chr_id_b37, position_b37, COUNT(*) as count FROM thegwensproject.genomiX.variants WHERE chr_id_b37 = '" +str(chr) +"' AND position_b37 = " + str(pos) + " GROUP BY chr_id_b37, position_b37 HAVING COUNT(*) > 1 LIMIT 20;")
    return query
query_job = client.query(count_variants(6, 166956723))
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row)

In [ ]:
def variant_info(chr, pos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.variants WHERE chr_id_b37 = '" +str(chr) +"' AND position_b37 = " + str(pos) + " LIMIT 20;")
    return query

query_job = client.query(variant_info(6, 166956723))
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row[9])
    print (row)

In [ ]:
def tag_disease_variant_info(chr, pos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.disease_variant_gene WHERE tag_chrom = '" +str(chr) +"' AND tag_pos = "+ str(pos) +"  LIMIT 20;")
    return query

query_job = client.query(tag_disease_variant_info(6, 166980468))
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row['lead_pos'])
    print (row)

In [75]:
def lead_disease_variant_info(chr, pos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.disease_variant_gene WHERE lead_chrom = '" +str(chr) +"' AND lead_pos = "+ str(pos) +"  LIMIT 20;")
    return query
query_job = client.query(lead_disease_variant_info(6, 166956723))
for row in query_job:
    # Row values can be accessed by field name or index.
    print (row)

In [ ]:
def exact_disease_variant_info(leadchr, leadpos, tagchr, tagpos):
    query = ( "SELECT * FROM thegwensproject.genomiX.disease_variant_gene WHERE lead_chrom = '" +str(leadchr) +"' AND lead_pos = "+ str(leadpos) +" AND tag_chrom = '" +str(tagchr) +"' AND tag_pos = "+ str(tagpos) +"  LIMIT 20;")
    return query
query_job = client.query(exact_disease_variant_info(6, 166980468, 6, 166952906))
for row in query_job:
    # Row values can be accessed by field name or index.
    print (row)
    print (row['trait_reported'])

In [ ]:
def lead_disease_info(chr, pos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.variant_disease WHERE lead_chrom = '" +str(chr) +"' AND lead_pos = "+ str(pos) +"  LIMIT 20;")
    return query
query_job = client.query(lead_disease_info(6, 166956723))
for row in query_job:
    # Row values can be accessed by field name or index.
    print (row['trait_reported'])

In [ ]:
def tag_disease_info(chr, pos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.variant_disease WHERE tag_chrom = '" +str(chr) +"' AND tag_pos = "+ str(pos) +"  LIMIT 20;")
    return query
query_job = client.query(tag_disease_info(6, 166952906))
for row in query_job:
    # Row values can be accessed by field name or index.
    print (row['trait_reported'])
    print (row['lead_chrom'] + " "+ str(row['lead_pos']))

In [ ]:
def exact_disease_info(leadchr, leadpos, tagchr, tagpos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.variant_disease WHERE tag_chrom = '" +str(tagchr) +"' AND tag_pos = "+ str(tagpos) +" AND lead_chrom = '" +str(leadchr) +"' AND lead_pos = "+ str(leadpos) +"  LIMIT 20;")
    return query
query_job = client.query(exact_disease_info(6, 166980468, 6, 166952906))
for row in query_job:
    # Row values can be accessed by field name or index.
    print (row['trait_reported'])

In [ ]:
def disease_catcher(disease):
    query = f'SELECT * FROM thegwensproject.genomiX.variant_disease WHERE trait_reported = "{disease}" LIMIT 20;'
    return query
query_job = client.query(disease_catcher("Alzheimer's disease"))
for row in query_job:
    # Row values can be accessed by field name or index.
    print (row['lead_chrom'] + " "+ str(row['lead_pos']))

In [ ]:
#use % of different parameters like acestry, for the disease specific genes.



In [7]:
import pandas as pd
import matplotlib.pyplot as plt

lead = pd.read_csv('lead_chrom_trait.csv')
tag = pd.read_csv('tag_chrom_trait.csv')
df_lead_pivot = lead.pivot(index='trait_reported', columns='lead_chrom')
df_tag_pivot = tag.pivot(index='trait_reported', columns='tag_chrom')
df_combined = df_lead_pivot.join(df_tag_pivot, lsuffix='_lead', rsuffix='_tag')

df_combined.reset_index(inplace=True)

#chrom_trait_joined = pd.merge(lead, tag, on='trait_reported', how='inner')


In [12]:
df_combined.fillna(0, inplace=True)
df_combined.head()

trait_reported count_lead  \
lead_chrom                                                             1   
0                 !Aliquae complicationes praecoces traumatis        0.0   
1           "Abnormal blood-pressure reading, without diag...        0.0   
2           "Abnormal findings on diagnostic imaging and i...      208.0   
3           "Abnormal findings on examination of blood, wi...        0.0   
4           "Abnormal findings on examination of other bod...        0.0   

                                                        ... count_tag       \
lead_chrom      10    11   12   13   14   15   16   17  ...        21   22   
0              0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...       0.0  0.0   
1           1157.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...       0.0  0.0   
2              0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...       0.0  0.0   
3              0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...       0.0  0.0   
4              0.0  33.0  0.0  0.0  0.0  0.0  0.0  0.0  ...       0.0  0.0   

                                                     
lead_chrom    3     4    5    6    7    8    9    X  
0           0.0  35.0  0.0  0.0  0.0  0.0  0.0  0.0  
1           0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2           0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3           0.0   0.0  0.0  0.0  0.0  9.0  0.0  0.0  
4           0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 47 columns]

In [13]:
df_combined['total'] = df_combined.iloc[:,1:].sum(axis=1)
df_percent = df_combined.copy()
for column in df_percent.columns[1:-1]:
    df_percent[column] = df_percent[column] / df_percent['total']*100
df_percent.drop('total', axis=1, inplace=True)
df_percent.head()

/tmp/ipykernel_67322/3530631317.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_percent.drop('total', axis=1, inplace=True)


trait_reported count_lead  \
lead_chrom                                                             1   
0                 !Aliquae complicationes praecoces traumatis        0.0   
1           "Abnormal blood-pressure reading, without diag...        0.0   
2           "Abnormal findings on diagnostic imaging and i...       50.0   
3           "Abnormal findings on examination of blood, wi...        0.0   
4           "Abnormal findings on examination of other bod...        0.0   

                                                      ... count_tag            \
lead_chrom    10    11   12   13   14   15   16   17  ...        21   22    3   
0            0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...       0.0  0.0  0.0   
1           50.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...       0.0  0.0  0.0   
2            0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...       0.0  0.0  0.0   
3            0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...       0.0  0.0  0.0   
4            0.0  50.0  0.0  0.0  0.0  0.0  0.0  0.0  ...       0.0  0.0  0.0   

                                                     
lead_chrom     4    5    6    7         8    9    X  
0           50.0  0.0  0.0  0.0  0.000000  0.0  0.0  
1            0.0  0.0  0.0  0.0  0.000000  0.0  0.0  
2            0.0  0.0  0.0  0.0  0.000000  0.0  0.0  
3            0.0  0.0  0.0  0.0  1.612903  0.0  0.0  
4            0.0  0.0  0.0  0.0  0.000000  0.0  0.0  

[5 rows x 47 columns]